In [ ]:
import pandas as pd
import py2neo
import tqdm



graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", ""), name="hw5")

targetFR = pd.read_csv("FR_target.csv")

transaction = """UNWIND $json as data CREATE (n:Streamer) SET n = data
"""

transaction_list = []

iteration = 0
for row in tqdm.tqdm(targetFR.iterrows()):
    transaction_list.append(dict(row[1]))
    iteration += 1
    if iteration % 1000 == 0:
        graph.run(transaction, json=transaction_list)
        transaction_list = []
        
graph.run(transaction, json=transaction_list)


query="""
LOAD CSV WITH HEADERS FROM 'file:///C:/Users/850/Desktop/Data_files/FR_edges.csv' AS csvLine 
MATCH (a:Streamer), (b:Streamer)   
where a.new_id=toInteger(csvLine.from) AND b.new_id=toInteger(csvLine.to)
CREATE (a)-[r:COLLAB]->(b)
"""
graph.run(query)

## question 1

In [ ]:
query= """MATCH (a:Streamer)-[r:COLLAB]-(b:Streamer)
RETURN DISTINCT a.new_id AS id, a.views AS Views ORDER BY Views DESC LIMIT 10
"""
result_1 = graph.run(query).to_data_frame()
result_1

## question 2


In [ ]:
query= """MATCH (n)
WITH n, size((n)-[:COLLAB->()) as degree
RETURN degree;
"""
import seaborn as sns

result_2 = graph.run(query).to_data_frame()
ax = sns.distplot(result_2["degree"],hist=False)
ax.title.set_text("Density of degrees")

# The power law can be used to describe a phenomenon where a small number of items is clustered at the top of a distribution.
# In this case we see that only the first 100 are at the top of the distribution. 

## question 3 

In [ ]:
query= """MATCH (n:Streamer)
WHERE n.mature = True
RETURN count(n.new_id)
"""
result_3 = graph.run(query).to_data_frame()
result_3

## question 4

In [ ]:
query= """MATCH (a:Streamer)-[r:COLLAB]->(b:Streamer)
WHERE a.mature=true
RETURN COUNT(DISTINCT a.new_id)
"""
result_4 = graph.run(query).to_data_frame()
result_4

## question 5

In [ ]:
query= """MATCH (a:Streamer)-[r:COLLAB]-(b:Streamer)
WHERE a.partner=true
RETURN COUNT( DISTINCT a.new_id)
"""
result_5_1 = graph.run(query).to_data_frame()
result_5_1


query= """MATCH (a:Streamer)-[r:COLLAB]-(b:Streamer)
WHERE a.partner=false
RETURN COUNT( DISTINCT a.new_id)
"""
result_5_2 = graph.run(query).to_data_frame()
result_5_2

## question 6

In [ ]:
# Sandbox exercise: Are the same streamrs at the to when calculated with views per day? 

query= """MATCH (a:Streamer)-[r:COLLAB]-(b:Streamer)
RETURN DISTINCT a.new_id AS id,a.days AS Days, a.views AS Views ,(a.views/a.days) as Views_per_day ORDER BY Views_per_day DESC LIMIT 10
"""
result_6_1 = graph.run(query).to_data_frame()
result_6_1

query= """MATCH (a:Streamer)-[r:COLLAB]-(b:Streamer)
RETURN DISTINCT a.new_id AS id, a.views AS Views ORDER BY Views DESC LIMIT 10
"""
result_6_2 = graph.run(query).to_data_frame()
result_6_2